# Metacritic All Time Games Statistics (1995 - 2018)

For this project, we used a dataset on Metacritic Games Reviews and Ratings from this kaggle page: https://www.kaggle.com/skateddu/metacritic-all-time-games-stats. Our goal is to ....


First, we import the necessary libraries:

In [40]:
# Import libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import numpy as np
import requests
import json
import requests
import pymongo
from scipy import stats 
import math

Next we import the dataset and preview the dataframe in pandas.

In [2]:
# Import dataset csv and preview
df = pd.read_csv('metacritic_games.csv')
df.head()

,name,platform,developer,publisher,genre,players,rating,attribute,release_date,link,critic_positive,critic_neutral,critic_negative,metascore,user_positive,user_neutral,user_negative,user_score
0,Command & Conquer,PC,Westwood Studios,Virgin Interactive,Sci-Fi,01-Apr,T,NaN,"Aug 31, 1995",/game/pc/command-conquer,5,0,0,94,47,0,1,8.9
1,Full Throttle,PC,LucasArts,LucasArts,Adventure,NaN,NaN,NaN,"Apr 30, 1995",/game/pc/full-throttle,6,2,0,86,18,1,0,8.7
2,Battle Arena Toshinden,PS,Tamsoft,SCEA,Action,01-Feb,T,NaN,"Sep 9, 1995",/game/playstation/battle-arena-toshinden,1,3,0,69,1,0,1,5.8
3,Sid Meier's Civilization II,PC,MPS Labs,MicroProse,Strategy,1 Player,K-A,NaN,"Feb 29, 1996",/game/pc/sid-meiers-civilization-ii,7,0,0,94,46,0,1,8.9
4,Quake,PC,id Software,id Software,Action,Jan-16,M,NaN,"Jun 22, 1996",/game/pc/quake,9,0,0,94,84,4,1,8.8


## Data Cleaning

### Removing incomplete rows and redundant columns

We begin data cleaning by dropping rows with missing values in developer and publisher.

In [3]:
# Drop rows with missing values in developer and publisher
df.dropna(subset=['developer', 'publisher'], inplace = True)
df.reset_index(inplace = True)
df = df.drop(['index'], axis = 1)

The following columns are dropped as they are not relevant to our research question:

In [4]:
# Drop redundant columns
df = df.drop(['players', 'attribute', 'link', 'critic_positive',
              'critic_neutral', 'critic_negative','user_positive',
              'user_neutral', 'user_negative'], axis = 1)

### Obtaining Release Year

Next, we sliced the 'release_date' to obtain the year of release and replaced the column with the appropriate 'release_year' data.

In [5]:
# Change Release Date to Release Year
year_released = []
for i in range(0,len(df['release_date'])):    
    year_released.append(df.release_date[i].split()[2])
    
df['release_date'] = year_released
df.rename(columns={"release_date": "release_year"}, inplace = True)

# Change datatype from str to integer
df[['release_year']] = df[['release_year']].astype(int)

### Rating

Rating columns are adjusted by replacing the letters with the actual rating description, as follows:
* **E** = Everyone
* **EC** = Early Childhood 3+
* **E10+** = Everyone 10+
* **T** = Teen 13+
* **M** = Mature 17+
* **AO** = Adults Only 18+
* **RP** or **K-A** = Rating Pending
* **Null values** are replaced as Rating Pending

In [6]:
# Add Description to Rating column and remove null values
for rating in df['rating']:
    if rating == 'E':
        df.rating.replace(to_replace = rating, value = 'Everyone', inplace=True)
    elif rating == 'EC':
        df.rating.replace(to_replace = rating, value = 'Early Childhood 3+', inplace=True)
    elif rating == 'E10+':
        df.rating.replace(to_replace = rating, value = 'Everyone 10+', inplace=True)
    elif rating == 'T':
        df.rating.replace(to_replace = rating, value = 'Teen 13+', inplace=True)
    elif rating == 'M':
        df.rating.replace(to_replace = rating, value = 'Mature 17+', inplace=True)
    elif rating == 'AO':
        df.rating.replace(to_replace = rating, value = 'Adults Only 18+', inplace=True)
    elif rating == 'RP' or rating == 'K-A' or type(rating) == float:
        df.rating.replace(to_replace = rating, value = 'Rating Pending', inplace=True)

### Adjusting User Score

The user_score column contains 0 null values, however 'tbd' is present, representing the user score is to be determined. As there are 2000 + values of 'tbd', we will replace the values with random-assigned values with weighted probabilities from other user review scores

In [7]:
# Assign 'tbd' values with random-assigned values with weighted probablities
choice = sorted(df[df.user_score != 'tbd']['user_score'].unique())
prob = df[df.user_score != 'tbd']['user_score'].value_counts(normalize=True).sort_index()
df['user_score'] = df.user_score.map(lambda x: np.random.choice(choice, 1, p= prob)[0] if x == 'tbd' else x)

For comparison purposes, we will adjust the user_score to be in the same format as the metascore by multiplying all values by 10 as user_score is rated on a scale of 10 while metascore is rated on a scale of 100. Appropriate datatype conversion will be performed to the values.

In [8]:
# Change datatype from str to float
df[['user_score']] = df[['user_score']].astype(float)

# Change user_score from a scale of 10 to a scale of 100 by multiplying all values by 10
df['user_score'] = df['user_score'].apply(lambda x: x * 10 )

# Change datatype from float to int
df[['user_score']] = df[['user_score']].astype(int)

In [9]:
print(df.shape)
df.head()

(20380, 9)


,name,platform,developer,publisher,genre,rating,release_year,metascore,user_score
0,Command & Conquer,PC,Westwood Studios,Virgin Interactive,Sci-Fi,Teen 13+,1995,94,89
1,Full Throttle,PC,LucasArts,LucasArts,Adventure,Rating Pending,1995,86,87
2,Battle Arena Toshinden,PS,Tamsoft,SCEA,Action,Teen 13+,1995,69,58
3,Sid Meier's Civilization II,PC,MPS Labs,MicroProse,Strategy,Rating Pending,1996,94,89
4,Quake,PC,id Software,id Software,Action,Mature 17+,1996,94,88


Great! We now have a clean dataframe in the right format with no missing values. The dataframe has a shape of **9 columns x 20380 rows**. We can now begin our data analysis and hypothesis testing.

## Data Analysis 

**PC vs Console over 2 range of years.**

In [58]:
df.user_score.groupby(by=df.platform).describe()

,count,mean,std,min,25%,50%,75%,max
platform,,,,,,,,
3DS,541.0,69.689464,13.019773,12.0,64.00,73.0,79.0,91.0
DC,143.0,77.069930,10.891688,7.0,74.00,80.0,83.0,91.0
DS,987.0,69.187437,14.728383,4.0,63.00,74.0,79.0,91.0
GBA,600.0,73.291667,13.316737,17.0,68.00,77.0,83.0,91.0
GC,512.0,74.271484,12.855456,13.0,70.00,78.0,83.0,92.0
N64,89.0,77.258427,12.748020,21.0,73.00,80.0,85.0,92.0
PC,5425.0,68.692350,13.879203,4.0,62.00,72.0,77.0,94.0
PS,256.0,75.324219,15.215329,4.0,68.75,80.0,87.0,92.0
PS2,1530.0,75.396078,12.884098,11.0,70.00,79.0,84.0,92.0


In [52]:
df.metascore.groupby(by=df.rating).describe()

,count,mean,std,min,25%,50%,75%,max
rating,,,,,,,,
Adults Only 18+,2.0,68.000000,35.355339,43.0,55.5,68.0,80.5,93.0
Early Childhood 3+,2.0,81.000000,7.071068,76.0,78.5,81.0,83.5,86.0
Everyone,5912.0,67.904939,13.689938,8.0,60.0,70.0,78.0,99.0
Everyone 10+,2896.0,68.395373,12.830613,17.0,61.0,70.0,78.0,97.0
Mature 17+,3480.0,70.723563,13.914907,13.0,63.0,73.0,81.0,98.0
Rating Pending,2307.0,69.566103,11.317592,11.0,64.0,71.0,78.0,94.0
Teen 13+,5781.0,68.591247,13.163696,12.0,61.0,70.0,78.0,98.0


To see if games with mature contents are received equally well by critics compared to games with appropriate contents for everyone.

* H0 : 'Mature 17+' and 'Everyone' games perform equally well for metascore
* H1 : 'Mature 17+' and "Everyone' games do not perform equally well for metascore

In [29]:
platform_boolean = []
for platform in df['platform']:
    if platform == 'PC':
        platform_boolean.append('PC')
    else:
        platform_boolean.append('Console')
df['platform_bool'] = platform_boolean

year_boolean = []
for year in df['release_year']:
    if year <= 2006:
        year_boolean.append('Old_12')
    elif year >= 2007:
        year_boolean.append('New_12')
df['year_bool'] = year_boolean

avg_score = []
for i in range(0,len(df)):
    avg_score.append((df.metascore[i] + df.user_score[i])/2)
df['avg_score'] = avg_score

score_indication = []
for score in df['avg_score']:
    if score >= 90:
        score_indication.append('Universal Acclaim')
    elif score >= 75 and score <= 89:
        score_indication.append('Favorable')
    elif score >= 50 and score <= 74:
        score_indication.append('Average')
    elif score >= 20 and score <= 49:
        score_indication.append('Unfavorable')
    else:
        score_indication.append('Overwhelming Dislike')
df['score_indication'] = score_indication 

In [30]:
df = df.reindex(columns=['name', 'platform', 'platform_bool', 'developer',
                         'publisher', 'genre', 'rating', 'release_year',
                        'year_bool', 'metascore', 'user_score', 'avg_score',
                        'score_indication'])
df

,name,platform,platform_bool,developer,publisher,genre,rating,release_year,year_bool,metascore,user_score,avg_score,score_indication
0,Command & Conquer,PC,PC,Westwood Studios,Virgin Interactive,Sci-Fi,Teen 13+,1995,Old_12,94,89,91.5,Universal Acclaim
1,Full Throttle,PC,PC,LucasArts,LucasArts,Adventure,Rating Pending,1995,Old_12,86,87,86.5,Favorable
2,Battle Arena Toshinden,PS,Console,Tamsoft,SCEA,Action,Teen 13+,1995,Old_12,69,58,63.5,Average
3,Sid Meier's Civilization II,PC,PC,MPS Labs,MicroProse,Strategy,Rating Pending,1996,Old_12,94,89,91.5,Universal Acclaim
4,Quake,PC,PC,id Software,id Software,Action,Mature 17+,1996,Old_12,94,88,91.0,Universal Acclaim
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20375,Shaq Fu: A Legend Reborn,PC,PC,Big Deez Productions,Saber Interactive,Action,Teen 13+,2018,New_12,30,67,48.5,Unfavorable
20376,ARK: Survival Evolved,Switch,Console,Studio Wildcard,Studio Wildcard,Action Adventure,Teen 13+,2018,New_12,29,24,26.5,Unfavorable
20377,The Quiet Man,PS4,Console,"Square Enix, Human Head Studios",Square Enix,Action Adventure,Mature 17+,2018,New_12,28,21,24.5,Unfavorable
20378,Awkward,Switch,Console,Snap Finger Click Ltd,Snap Finger Click Ltd,Miscellaneous,Mature 17+,2018,New_12,28,79,53.5,Average


In [47]:
df.genre.value_counts()

Action              7100
Action Adventure    2265
Sports              1841
Role-Playing        1787
Strategy            1645
                    ... 
Rail                   1
Other                  1
Pinball                1
Formula One            1
Wargame                1
Name: genre, Length: 68, dtype: int64

In [18]:
df.platform_bool.groupby(by=df.year_bool).value_counts()

year_bool  platform_bool
New_12     Console          10634
           PC                3902
Old_12     Console           4321
           PC                1523
Name: platform_bool, dtype: int64

In [37]:
df.score_indication.groupby(by=df.rating).describe()

,count,unique,top,freq
rating,,,,
Adults Only 18+,2,2,Unfavorable,1
Early Childhood 3+,2,2,Average,1
Everyone,5912,5,Average,3392
Everyone 10+,2896,5,Average,1702
Mature 17+,3480,5,Average,1723
Rating Pending,2307,5,Average,1421
Teen 13+,5781,5,Average,3138


In [36]:
df.user_score.groupby(by=df.genre).describe().sort_values(by = 'count', ascending = False).head(10)

,count,mean,std,min,25%,50%,75%,max
genre,,,,,,,,
Action,7100.0,68.809437,14.323006,4.0,62.0,72.0,79.0,92.0
Action Adventure,2265.0,69.935541,14.370762,10.0,63.0,73.0,80.0,92.0
Sports,1841.0,68.948941,14.541754,7.0,62.0,73.0,79.0,89.0
Role-Playing,1787.0,72.665921,12.339849,16.0,67.0,75.0,82.0,93.0
Strategy,1645.0,70.841945,13.605873,12.0,65.0,74.0,80.0,92.0
Miscellaneous,1629.0,69.751381,13.822459,8.0,64.0,73.0,80.0,91.0
Adventure,1169.0,67.918734,13.207488,8.0,62.0,70.0,77.0,91.0
Driving,1114.0,70.220826,14.097519,4.0,63.0,74.0,80.0,90.0
Simulation,631.0,68.057052,15.119521,1.0,62.0,72.0,78.0,89.0


In [34]:
df.rating.groupby(by=df.score_indication).describe()

,count,unique,top,freq
score_indication,,,,
Average,11377,6,Everyone,3392
Favorable,7044,6,Teen 13+,2104
Overwhelming Dislike,527,5,Teen 13+,154
Unfavorable,1264,6,Everyone,388
Universal Acclaim,168,6,Mature 17+,75


In [35]:
df.genre.groupby(by=df.score_indication).describe().sort_values(by = 'count', ascending = False).head(10)

,count,unique,top,freq
score_indication,,,,
Average,11377,58,Action,4055
Favorable,7044,43,Action,2286
Unfavorable,1264,28,Action,531
Overwhelming Dislike,527,20,Action,174
Universal Acclaim,168,13,Action,54
